In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
cip_dev = pd.read_csv('raw data\cip_dataset_v3.csv')
cip_dev = cip_dev.loc[cip_dev['currency'] == 'EUR']
cip_dev['date'] = pd.to_datetime(cip_dev['date'], format='%d%b%Y', errors='coerce')
cip_dev['cip_govt'] = cip_dev['cip_govt'].interpolate(method='linear')
cip_dev = cip_dev[cip_dev['date'] >= '2008-01-01']
cip_dev = cip_dev[['date', 'cip_govt']]
cip_dev.head()

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\mathi\AppData\Local\Temp\ipykernel_53260\3319912346.py:1: SyntaxWarning: invalid escape sequence '\c'
  cip_dev = pd.read_csv('raw data\cip_dataset_v3.csv')


,date,cip_govt
782557,2008-01-01,18.146667
782558,2008-01-02,17.330000
782559,2008-01-03,17.870000
782560,2008-01-04,13.930000
782561,2008-01-07,13.115000


In [3]:
fed_target = pd.read_csv('raw data\DFEDTAR.csv')
fed_target_u = pd.read_csv('raw data\DFEDTARU.csv')
fed_target_l = pd.read_csv('raw data\DFEDTARL.csv')


fed_target['observation_date'] = pd.to_datetime(fed_target['observation_date'])
fed_target_u['observation_date'] = pd.to_datetime(fed_target_u['observation_date'])
fed_target_l['observation_date'] = pd.to_datetime(fed_target_l['observation_date'])

merged_fed_target = pd.merge(fed_target, fed_target_u, on='observation_date', how='outer', suffixes=('', '_u'))
merged_fed_target = pd.merge(merged_fed_target, fed_target_l, on='observation_date', how='outer', suffixes=('', '_l'))

merged_fed_target['fed_target'] = merged_fed_target.apply(
    lambda row: row['DFEDTAR'] if pd.notna(row['DFEDTAR']) and row['observation_date'] <= pd.Timestamp('2008-12-15')
    else (row['DFEDTARU'] + row['DFEDTARL']) / 2 if pd.notna(row['DFEDTARU']) and pd.notna(row['DFEDTARL']) #assume that the Fed target rate is the average of the upper and lower bound of the corridor
    else None,
    axis=1)

merged_fed_target = merged_fed_target.rename(columns={'observation_date': 'date'})
merged_fed_target = merged_fed_target[['date', 'fed_target']]
print(merged_fed_target)

           date  fed_target
0    2008-01-01       4.250
1    2008-01-02       4.250
2    2008-01-03       4.250
3    2008-01-04       4.250
4    2008-01-05       4.250
...         ...         ...
6215 2025-01-06       4.375
6216 2025-01-07       4.375
6217 2025-01-08       4.375
6218 2025-01-09       4.375
6219 2025-01-10       4.375

[6220 rows x 2 columns]


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\mathi\AppData\Local\Temp\ipykernel_53260\3629607993.py:1: SyntaxWarning: invalid escape sequence '\D'
  fed_target = pd.read_csv('raw data\DFEDTAR.csv')
C:\Users\mathi\AppData\Local\Temp\ipykernel_53260\3629607993.py:2: SyntaxWarning: invalid escape sequence '\D'
  fed_target_u = pd.read_csv('raw data\DFEDTARU.csv')
C:\Users\mathi\AppData\Local\Temp\ipykernel_53260\3629607993.py:3: SyntaxWarning: invalid escape sequence '\D'
  fed_target_l = pd.read_csv('raw data\DFEDTARL.csv')


In [4]:
eur_usd = pd.read_csv('raw data/EUR_USD.csv')
eur_usd['Date'] = pd.to_datetime(eur_usd['Date'])
eur_usd.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis=1, inplace=True)
eur_usd.rename(columns={'Price': 'EUR/USD', 'Date': 'date'}, inplace=True)
print(eur_usd.head())
print(eur_usd.dtypes)

vix = pd.read_csv('raw data/VIX.csv')
vix['Date'] = pd.to_datetime(vix['Date'], format='mixed')
vix = vix[['Date', 'Dernier']]
vix.rename(columns={'Dernier': 'VIX', 'Date': 'date'}, inplace=True)
vix['VIX'] = vix['VIX'].str.replace(',', '.').astype(float)
vix.sort_values(by='date', inplace=True)
print(vix.head())
print(vix.dtypes)

us_tb90d= pd.read_csv('raw data\TB3MS.csv')
us_tb90d.rename(columns={'observation_date': 'date', 'TB3MS': 'us_tb90d'}, inplace=True)
us_tb90d['date'] = pd.to_datetime(us_tb90d['date'])
us_tb90d.head()
print(us_tb90d.dtypes)

ecb_fdr = pd.read_csv('raw data/ECB_FDR.csv')
ecb_fdr['DATE'] = pd.to_datetime(ecb_fdr['DATE'])
ecb_fdr = ecb_fdr.iloc[:, [0,2]]
ecb_fdr.columns.values[0], ecb_fdr.columns.values[1] = 'date', 'ecb_fdr'
print(ecb_fdr.head())
print(ecb_fdr.dtypes)

euribor = pd.read_csv('raw data/Euribor_90d.csv')
euribor['DATE'] = pd.to_datetime(euribor['DATE'])
euribor = euribor.iloc[:, [0,2]]
euribor.rename(columns = {'DATE': 'date', euribor.columns.values[1]: 'Euribor_90d'}, inplace=True)
euribor = euribor[euribor['date'] >= '2008-01-01']
print(euribor.head())
print(euribor.dtypes)


        date  EUR/USD
0 2024-12-18   1.0501
1 2024-12-17   1.0489
2 2024-12-16   1.0510
3 2024-12-13   1.0502
4 2024-12-12   1.0467
date       datetime64[ns]
EUR/USD           float64
dtype: object
           date    VIX
4317 2007-12-31  22.50
4295 2008-01-02  24.02
4255 2008-01-04  22.68
4233 2008-01-05  18.88
4191 2008-01-07  23.65
date    datetime64[ns]
VIX            float64
dtype: object
date        datetime64[ns]
us_tb90d           float64
dtype: object
        date  ecb_fdr
0 1999-01-01     2.00
1 1999-01-02     2.00
2 1999-01-03     2.00
3 1999-01-04     2.75
4 1999-01-05     2.75
date       datetime64[ns]
ecb_fdr           float64
dtype: object
          date  Euribor_90d
168 2008-01-31       4.4815
169 2008-02-29       4.3621
170 2008-03-31       4.5964
171 2008-04-30       4.7835
172 2008-05-31       4.8574
date           datetime64[ns]
Euribor_90d           float64
dtype: object


<>:17: SyntaxWarning: invalid escape sequence '\T'
<>:17: SyntaxWarning: invalid escape sequence '\T'
C:\Users\mathi\AppData\Local\Temp\ipykernel_53260\1843354132.py:17: SyntaxWarning: invalid escape sequence '\T'
  us_tb90d= pd.read_csv('raw data\TB3MS.csv')


In [5]:
ecb_al = pd.read_csv('raw data/ECB_Assets_liabilities.csv')
ecb_al['DATE'] = pd.to_datetime(ecb_al['DATE'])
ecb_al.drop(['TIME PERIOD'], axis=1, inplace=True)
ecb_al.rename(columns = {"DATE": "date", "Total Assets/Liabilities reported by the ECB (stocks) (BSI.M.4F.N.N.T00.A.1.Z5.0000.Z01.E)":"ECB_assets"}, inplace=True)

last_known_row = ecb_al[ecb_al['date'] <= pd.to_datetime('01/01/2008')].iloc[-1]
new_row = last_known_row.copy()
new_row['date'] = pd.to_datetime('01/01/2008')
ecb_al.loc[len(ecb_al)] = new_row
ecb_al = ecb_al[ecb_al['date'] >= '2008-01-01']
ecb_al = ecb_al.sort_values('date')

print(ecb_al.head())
print(ecb_al.dtypes)

          date  ECB_assets
310 2008-01-01    254025.0
108 2008-01-31    281242.0
109 2008-02-29    280272.0
110 2008-03-31    286875.0
111 2008-04-30    299684.0
date          datetime64[ns]
ECB_assets           float64
dtype: object


In [6]:
fed_al = pd.read_csv('raw data/Fed_WALCL.csv')
fed_al['observation_date'] = pd.to_datetime(fed_al['observation_date'])
fed_al.rename(columns = {"observation_date": "date", "WALCL": "FED_assets"}, inplace=True)

last_known_row = fed_al[fed_al['date'] <= pd.to_datetime('01/01/2008')].iloc[-1]
new_row = last_known_row.copy()
new_row['date'] = pd.to_datetime('01/01/2008')
fed_al.loc[len(fed_al)] = new_row
fed_al = fed_al[fed_al['date'] >= '2008-01-01']
fed_al = fed_al.sort_values('date')

print(fed_al.head())
print(fed_al.dtypes)

           date  FED_assets
1148 2008-01-01      890662
263  2008-01-02      922187
264  2008-01-09      880754
265  2008-01-16      893581
266  2008-01-23      877253
date          datetime64[ns]
FED_assets             int64
dtype: object


In [7]:
dfs = [ecb_al, fed_al, eur_usd, vix, euribor]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)
merged_df = merged_df.sort_values(by='date')
print(merged_df.head()) 

        date  ECB_assets  FED_assets  EUR/USD    VIX  Euribor_90d
0 2007-12-31         NaN         NaN      NaN  22.50          NaN
1 2008-01-01    254025.0    890662.0   1.4587    NaN          NaN
2 2008-01-02         NaN    922187.0   1.4725  24.02          NaN
3 2008-01-03         NaN         NaN   1.4746    NaN          NaN
4 2008-01-04         NaN         NaN   1.4743  22.68          NaN


In [8]:
merged_df[['ECB_assets', 'FED_assets', 'EUR/USD', 'VIX']] = merged_df[['ECB_assets', 'FED_assets', 'EUR/USD', 'VIX']].ffill()
merged_df['Euribor_90d'] = merged_df['Euribor_90d'].bfill()
merged_df['VIX'] = merged_df['VIX'].interpolate()
merged_df = merged_df[(merged_df['date'] >= '2008-01-01') & (merged_df['date'] <= '2024-12-01')]
print(merged_df.head(), "\n", merged_df.dtypes)


        date  ECB_assets  FED_assets  EUR/USD    VIX  Euribor_90d
1 2008-01-01    254025.0    890662.0   1.4587  22.50       4.4815
2 2008-01-02    254025.0    922187.0   1.4725  24.02       4.4815
3 2008-01-03    254025.0    922187.0   1.4746  24.02       4.4815
4 2008-01-04    254025.0    922187.0   1.4743  22.68       4.4815
5 2008-01-05    254025.0    922187.0   1.4743  18.88       4.4815 
 date           datetime64[ns]
ECB_assets            float64
FED_assets            float64
EUR/USD               float64
VIX                   float64
Euribor_90d           float64
dtype: object


In [9]:
print(merged_fed_target.head())

        date  fed_target
0 2008-01-01        4.25
1 2008-01-02        4.25
2 2008-01-03        4.25
3 2008-01-04        4.25
4 2008-01-05        4.25


In [10]:
other_dfs = [cip_dev, merged_fed_target, us_tb90d]

for df in other_dfs:
    print(df.head())
    merged_df = pd.merge(merged_df, df, on='date', how='outer')

print(merged_df.head())

             date   cip_govt
782557 2008-01-01  18.146667
782558 2008-01-02  17.330000
782559 2008-01-03  17.870000
782560 2008-01-04  13.930000
782561 2008-01-07  13.115000
        date  fed_target
0 2008-01-01        4.25
1 2008-01-02        4.25
2 2008-01-03        4.25
3 2008-01-04        4.25
4 2008-01-05        4.25
        date  us_tb90d
0 2008-01-01      2.75
1 2008-02-01      2.12
2 2008-03-01      1.26
3 2008-04-01      1.29
4 2008-05-01      1.73
        date  ECB_assets  FED_assets  EUR/USD   VIX  Euribor_90d    cip_govt  \
0 2008-01-01    254025.0    890662.0   1.4587  22.5       4.4815   18.146667   
1 2008-01-01    254025.0    890662.0   1.4587  22.5       4.4815   28.500000   
2 2008-01-01    254025.0    890662.0   1.4587  22.5       4.4815   38.662500   
3 2008-01-01    254025.0    890662.0   1.4587  22.5       4.4815  102.233268   
4 2008-01-01    254025.0    890662.0   1.4587  22.5       4.4815   48.375000   

   fed_target  us_tb90d  
0        4.25      2.75  
1    

In [13]:
merged_df.to_csv('daily_data.csv', index=False)